In [112]:
pd.options.display.max_rows = 4000

In [4]:
from tabula import read_pdf
import tabula
import pandas as pd
import re
import math

In [11]:
def pdf_transform_v1(start, end, pdf):
    
    unit = []
    station = []
    line = []
    availability = []
    comment = ''
    comments = []
    
    df = tabula.read_pdf(pdf, pages=list(range(start, end + 1)))
    df.dropna(how = 'all', inplace = True)
    df.reset_index(inplace = True, drop = True)
    
    df_revision = df.copy()
    df_null = df.isna()
    
    df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)
    
    df_comment = pd.DataFrame(
        {'full comment': df['Unnamed: 3'].str.contains('^(Th).*$'),
         'starts percent': df['Unnamed: 3'].str.contains('\d{1,2}\.\d{2}\%'),
         'continued comment': df['Unnamed: 3'].str.contains('^[a-zA-SU-Z]*[\s]'),
         'all caps': df['Unnamed: 3'].str.contains('^[^a-z]*$'),
         'no spaces': df['Unnamed: 3'].str.contains('^[^\s]*$'),
        })

    for i in range(len(df)):
        try:
            if math.isnan(float(df.iloc[i][1])) is True:
                pass
        except:
            if df.iloc[i][1] != 'Location':
                unit.append(df.iloc[i][0])
                station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
                line.append(df.iloc[i][1].split(' ')[-1])
        if df.iloc[i][1] != 'Location':
            if df.iloc[i][2] != 'Location':
                if df.iloc[i].isna().sum() == 0:
                    availability.append(df.iloc[i][2])
                if df.iloc[i].isna().sum() == 1:
                    availability.append(df.iloc[i][3].split(' ')[0])
        if df_comment['starts percent'][i] == True:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comment += ' '.join(df_revision['Unnamed: 3'][i].split(' ')[1:])

        if df_comment['continued comment'][i] == True and df_null['Unnamed: 2'][i] == True:
            comment += df_revision['Unnamed: 3'][i]

        if df_comment['full comment'][i] == True:
            if len(comment) > 0:
                comments.append(comment)
                comment = ''
            comments.append(df_revision['Unnamed: 3'][i])

    df_stations = pd.DataFrame(
        {'unit': unit,
         'station': station,
         'line' : line,
         'availability' : availability,
         'comments' : comments
        })
    
    return df_stations

In [12]:
def pdf_transform_v2(start, end, pdf):

    df = tabula.read_pdf(pdf, pages=start)

    for i in range(len(df.columns)):
        df.rename(columns = {df.columns[i]:str(i)}, inplace = True) 

    for i in range(start + 1, end):
        df2 = tabula.read_pdf(pdf, pages=[i])
        for i in range(len(df2.columns)):
            df2.rename(columns = {df2.columns[i]:str(i)}, inplace = True) 

        df = pd.concat([df, df2], axis=0).reset_index(drop = True)

    df.dropna(how = 'all', inplace = True)
    df.reset_index(inplace = True, drop = True)

    unit = []
    station = []
    line = []
    availability = []
    comment = ''
    comments = []

    if len(df.columns) > 4:
        df = df[list(df.columns[:-1])]

    df_revision = df.copy()
    df_null = df.isna()

    df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

    df_comment = pd.DataFrame(
        {'full comment': df['3'].str.contains('^(Th).*$'),
         'starts percent': df['3'].str.contains('\d{1,2}\.\d{2}\%'),
         'continued comment': df['3'].str.contains('^[a-zA-SU-Z]*[\s]'),
         'all caps': df['3'].str.contains('^[^a-z]*$'),
         'no spaces': df['3'].str.contains('^[^\s]*$'),
        })

    for i in range(len(df)):
        try:
            if math.isnan(float(df.iloc[i][1])) is True:
                pass
        except:
            if df.iloc[i][1] != 'Location':
                if re.match(string = df.iloc[i][0], pattern = 'E|[SL]\d{3}'):
                    unit.append(df.iloc[i][0])
                station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
                line.append(df.iloc[i][1].split(' ')[-1])
            if df.iloc[i].isna().sum() == 0:
                if re.match(string = df.iloc[i][2], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][2])
            if df.iloc[i].isna().sum() == 1:
                if re.match(string = df.iloc[i][3].split(' ')[0], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][3].split(' ')[0])
            if df_comment['starts percent'][i] == True:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comment += ' '.join(df_revision['3'][i].split(' ')[1:])

            if df_comment['continued comment'][i] == True and df_null['2'][i] == True:
                comment += df_revision['3'][i]

            if df_comment['full comment'][i] == True:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comments.append(df_revision['3'][i])

    df_stations = pd.DataFrame(
        {'unit': unit,
         'station': station,
         'line' : line,
         'availability' : availability,
         'comments' : comments
        })

    return df_stations

In [78]:
pdf_transform_v2(380, 384, '190923_1000_Transit.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL204,Grand Central-42 St,4567S,72.36%,This elevator was out of service on multiple d...
1,EL119,181 St,A,81.16%,This elevator was out of service from 6/3/19 t...
2,EL224,8 Av,L,83.42%,This elevator was out of service on multiple o...
3,ES104,Gun Hill Rd,25,74.40%,This escalator had multiple outages for lower ...
4,ES105,Gun Hill Rd,25,79.55%,This escalator was out of service multiple tim...


In [79]:
pdf_transform_v2(235, 238, '190520_1030_Transit.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL340,Franklin Av,CS,78.96%,This elevator had 2 incidents and it was out o...
1,EL312,Clark St,23,81.47%,This elevator was out of service 8 times due t...
2,ES104,Gun Hill Rd,25,82.89%,This escalator was out of service from 1/7/19 ...
3,ES106,West Farms Sq-E Tremont Av,25,77.55%,This escalator was out of service from 12/28/1...
4,ES108,Intervale Av,25,78.18%,This escalator was out of service from 12/29/1...


In [80]:
pdf_transform_v2(328, 330, '190225_1030_transit-bus.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


ValueError: arrays must all be same length

In [225]:
def pdf_transform_v3(start, end, pdf):

    df = tabula.read_pdf(pdf, pages=start)

    for i in range(len(df.columns)):
        df.rename(columns = {df.columns[i]:str(i)}, inplace = True) 

    for i in range(start + 1, end):
        df2 = tabula.read_pdf(pdf, pages=[i])
        for i in range(len(df2.columns)):
            df2.rename(columns = {df2.columns[i]:str(i)}, inplace = True) 

        df = pd.concat([df, df2], axis=0).reset_index(drop = True)

    df.dropna(how = 'all', inplace = True)
    df.reset_index(inplace = True, drop = True)

    unit = []
    station = []
    line = []
    availability = []
    comment = ''
    comments = []

    if len(df.columns) > 4:
        df = df[list(df.columns[:-1])]

    df_revision = df.copy()
    df_null = df.isna()

    df_revision.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=[" "," "], regex=True, inplace=True)

    df_comment = pd.DataFrame(
        {'full comment': df['3'].str.contains('^(Th).*$'),
         'starts percent': df['3'].str.contains('\d{1,2}\.\d{2}\%'),
         'continued comment': df['3'].str.contains('^[a-zA-SU-Z]*[\s]'),
         'all caps': df['3'].str.contains('^[^a-z]*$'),
         'no spaces': df['3'].str.contains('^[^\s]*$'),
        })

    for i in range(len(df)):
        try:

            if math.isnan(float(df.iloc[i][1])) is True:
                pass

        except:

            if df.iloc[i][1] != 'Location':
                if re.match(string = df.iloc[i][0], pattern = 'E|[SL]\d{3}'):
                    unit.append(df.iloc[i][0])
                station.append(' '.join(df.iloc[i][1].split(' ')[:-1]))
                line.append(df.iloc[i][1].split(' ')[-1])

            if df.iloc[i].isna().sum() == 0:
                if re.match(string = df.iloc[i][2], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][2])

            if df.iloc[i].isna().sum() == 1:
                if re.match(string = df.iloc[i][3].split(' ')[0], pattern = '\d{1,2}\.\d{2}\%'):
                    availability.append(df.iloc[i][3].split(' ')[0])

            if df_comment['starts percent'][i] == True:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comment += ' '.join(df_revision['3'][i].split(' ')[1:])

            if df_comment['full comment'][i] == True and df_null['2'][i] == True:
                comment += df_revision['3'][i]

            if df_comment['full comment'][i] == True and df_null['2'][i] == False:
                if len(comment) > 0:
                    comments.append(comment)
                    comment = ''
                comments.append(df_revision['3'][i])

            if df_comment['continued comment'][i] == True:
                comment += df_revision['3'][i]

    if len(comment) > 0:
        comments.append(comment)

    df_stations = pd.DataFrame(
        {'unit': unit,
         'station': station,
         'line' : line,
         'availability' : availability,
         'comments' : comments
        })
    
    return df_stations

In [227]:
pdf_transform_v3(328, 330, '190225_1030_transit-bus.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL120,190 St,A,27.18%,This elevator was out multiple times due to le...
1,EL249,86 St,Q,84.06%,This elevator was out of service from 11/19/18...
2,ES104,Gun Hill Rd,25,83.76%,From 10/1/18 to 12/23/18 the escalator went ou...
3,ES105,Gun Hill Rd,25,83.48%,From 10/1/18 to 12/23/18 the escalator went ou...
4,ES106,West Farms Sq-E Tremont Av,25,79.51%,From 10/1/18 to 12/23/18 the escalator went ou...


In [228]:
pdf_transform_v3(235, 238, '190520_1030_Transit.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL340,Franklin Av,CS,78.96%,This elevator had 2 incidents and it was out o...
1,EL312,Clark St,23,81.47%,This elevator was out of service 8 times due t...
2,ES104,Gun Hill Rd,25,82.89%,This escalator was out of service from 1/7/19 ...
3,ES106,West Farms Sq-E Tremont Av,25,77.55%,This escalator was out of service from 12/28/1...
4,ES108,Intervale Av,25,78.18%,This escalator was out of service from 12/29/1...


In [229]:
pdf_transform_v3(380, 384, '190923_1000_Transit.pdf').head()

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,unit,station,line,availability,comments
0,EL204,Grand Central-42 St,4567S,72.36%,This elevator was out of service on multiple d...
1,EL119,181 St,A,81.16%,This elevator was out of service from 6/3/19 t...
2,EL224,8 Av,L,83.42%,This elevator was out of service on multiple o...
3,ES104,Gun Hill Rd,25,74.40%,This escalator had multiple outages for lower ...
4,ES105,Gun Hill Rd,25,79.55%,This escalator was out of service multiple tim...


In [230]:
pdf_transform_v3(213, 214, '181113_1030_Transit.pdf').head()

KeyError: '3'

In [231]:
df

,0,1,2,3
0,Borough/ Unit,Location,24 Hr\rAvailability,Comments
1,MANHATTAN,NaN,NaN,NaN
2,EL120,190 St A,27.18%,This elevator was out multiple times due to le...
3,EL249,86 St Q,84.06%,This elevator was out of service from 11/19/18...
4,Borough/ Unit,NaN,Location,24 Hr Comments
5,NaN,NaN,NaN,Availability
6,BRONX,NaN,NaN,NaN
7,ES104,Gun Hill Rd 25,NaN,83.76% From 10/1/18 to 12/23/18 the escalator ...
8,NaN,NaN,NaN,activation due to vandalism. From 10/1/18 to 1...
9,NaN,NaN,NaN,to the comb stop safety device being too sensi...
